In [ ]:
from __future__ import print_function
import KerasTools as KT
import numpy as np
from keras.datasets import mnist
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical

In [ ]:
# Load MNIST data and preprocess it
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Generate a neural network
#
# Here we will use SGD for smoother training curves.
# Usually one would rather use an adaptive optimizer like adam or rmsprop

def build_network():
    network = models.Sequential()
    network.add(layers.Dense(64, activation='relu', input_shape=(28*28*1,)))
    network.add(layers.Dense(64, activation='relu'))
    network.add(layers.Dense(10, activation='softmax'))

    network.compile(optimizer=optimizers.sgd(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
# Build a fresh new neural network and plot its architecture
network = build_network()
network.summary()

In [ ]:
# Pass 1:
#
# Train the network with the MNIST training set for 50 epochs and a batch size of 128
# Use this time 20% of training data to validate training progress

epochs = 50
history = network.fit(train_images, train_labels, epochs=epochs, batch_size=128, validation_split=0.2)

In [ ]:
KT.plot_history(history.history)

In [ ]:
# Pass 2:
#
# Use the 1st pass results to determine the optimal number of epochs where validation loss is at its minimum.
# Train a new network a second time with the whole training dataset
network = build_network()
epochs = 20
network.fit(train_images, train_labels, epochs=epochs, batch_size=128)

# After final training, evaluate the generalizing power of the network with the test set. 
test_loss, test_acc = network.evaluate(test_images, test_labels)
print()
print("Test loss: {}".format(test_loss))
print("Test accuracy: {}".format(test_acc))
history.history['test_loss'] = test_loss
history.history['test_acc'] = test_acc
history.history['epochs'] = epochs

In [ ]:
# Plot loss/accuracy training history and test results in some nice graphs
KT.plot_history(history.history)

In [ ]:
# Save the network for later production use
network.save("./mnist_trained.h5")